<a href="https://colab.research.google.com/github/parasgulati8/Web-Scraper/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import glob
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# import urllib3
import requests
# import json
# from urllib.request import urlopen, urlretrieve, quote
# from openpyxl import Workbook
import re

In [0]:
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
parcel = soup.h2.text[34:]
# parcel = ' '.join(parcel.split()[6:])
df = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})[0]

df['Parcel'] = [parcel]*len(df)
df

,Sale Date,Sale Price,Deed Type,Deed Book & Page,Parcel
0,12/05/2006,"$7,500",DEED OF CORRECTION,20110805-0060547,001 00 0 001.00
1,12/05/2006,"$7,500",WARRANTY DEED,20070109-0003943,001 00 0 001.00
2,09/15/1965,$0,WARRANTY DEED,00000137-0000216,001 00 0 001.00


In [2]:
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
parcel = soup.h2.text
parcel = ' '.join(parcel.split()[6:])
df = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})[0]

df['Parcel'] = [parcel]*len(df)
df['Property #'] = [1]*len(df)
i = 2
while True:
  url = 'http://www.padctn.org/prc/property/' + str(i) + '/card/' + str(i) + '/historical'
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  parcel = soup.h2.text[34:]
  # parcel = ' '.join(parcel.split()[6:])
  
  dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                        attrs={"class":"table table-condensed table-bordered"})
  dfs[0]['Parcel'] = [parcel]*len(dfs[0])
  dfs[0]['Property #'] = [i]*len(dfs[0])


  print(i, end=' ')
  if np.any(dfs[0]) or np.any(dfs[1]):
    df = pd.concat([df, dfs[0]] )
    i = i+1
  else:
    print ('Table Empty, Exiting...')
    print (table)
    break
  if i % 10 == 1:
    print()
  # time.sleep(3)
  if i == 5:
    break

2 3 4 

In [0]:
df.reset_index(inplace=True)

In [6]:
df

,index,Sale Date,Sale Price,Deed Type,Deed Book & Page,Parcel,Property #
0,0,12/05/2006,"$7,500",DEED OF CORRECTION,20110805-0060547,001 00 0 001.00,1
1,1,12/05/2006,"$7,500",WARRANTY DEED,20070109-0003943,001 00 0 001.00,1
2,2,09/15/1965,$0,WARRANTY DEED,00000137-0000216,001 00 0 001.00,1
3,0,05/17/1983,"$12,500",WARRANTY DEED,00006058-0000985,001 00 0 002.00,2
4,1,01/06/1972,$0,WARRANTY DEED,00005797-0000390,001 00 0 002.00,2
5,2,01/06/1972,$0,WARRANTY DEED,00000155-0000282,001 00 0 002.00,2
6,0,12/18/1997,"$67,000",WARRANTY DEED,00010729-0000778,002 00 0 001.00,3
7,1,07/29/1997,"$50,000",WARRANTY DEED,00010552-0000735,002 00 0 001.00,3
8,2,04/13/1994,"$8,000",WARRANTY DEED,00009312-0000366,002 00 0 001.00,3
9,3,10/27/1977,"$23,000",WARRANTY DEED,00005215-0000894,002 00 0 001.00,3
